# Processing lineP data from waterproperties. 
The data was downloaded from www.waterproperties.ca as .ctd files. we used the LineP area which inlcudes data in between stations. This script will process the raw data into a dataframe and save it as a csv.

The data was extracted from the .ctd files to netcdf using the quality control flags (0 AND 3). This script loads these in to create a dataframe for each year and subsamples by excluding any data that is far from the stations. We include profiles that are within 10km of a station, or 25km for station P26. If the file does not contain pressure, we use the depth coordinate and convert to pressure using p=gsw.conversions.p_from_z(-d, lat).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from scipy.spatial import cKDTree
from pyproj import Geod
import xarray as xr
import os
import glob
from datetime import datetime, timedelta

ERROR 1: PROJ: proj_create_from_database: Open of /home/amh001/space_fs7/software_2022/python/py_2024/share/proj failed


In [2]:
# %%
def load_temperature_profiles(file_paths):
    """
    Load profiles including temperature, salinity, pressure, and oxygen variables from multiple NetCDF files.
    
    Parameters:
        file_paths (list of str): List of paths to NetCDF files.
    
    Returns:
        pd.DataFrame: Combined DataFrame with time index and columns for filename, pressure, temperature,
                      salinity, latitude, longitude, and oxygen variables.
    """
    oxygen_mapping = {
        'oxygen': 'OXYGEN_MMOL_M3',
        'oxy_umol_kg': 'OXYGEN_UMOL_KG'
    }
    
    all_data = []
    
    for path in file_paths:
        try:
            ds = xr.open_dataset(path, decode_cf=True)
            cast_time = pd.to_datetime(ds['time'].values[0])
    
            # Always require temperature, salinity, and pressure
            temperature = ds['temperature'].values
            salinity = ds['salinity'].values
            pressure = ds['Pres'].values
            filename = os.path.basename(path)
    
            # Ensure pressure is 1D
            if pressure.ndim > 1:
                pressure = pressure.flatten()
            
            # Prepare dictionary with always-present variables
            var_data = {
                'time': cast_time,
                'file': filename,
                'CTDPRS_DBAR': pressure,
                'CTDTMP_ITS90_DEG_C': temperature,
                'SALINITY_PSS78': salinity
            }
    
            # Extract latitude and longitude (if available; if not, use NaN)
            var_data['latitude'] = ds['latitude'].values.item() if 'latitude' in ds else np.nan
            var_data['longitude'] = ds['longitude'].values.item() if 'longitude' in ds else np.nan
    
            # Always include oxygen variables
            for netcdf_var, output_col in oxygen_mapping.items():
                if netcdf_var in ds and ds[netcdf_var].values.size > 0:
                    data = ds[netcdf_var].values
                    if not np.all(np.isnan(data)):
                        var_data[output_col] = data
                    else:
                        var_data[output_col] = np.full(len(pressure), np.nan, dtype=float)
                else:
                    var_data[output_col] = np.full(len(pressure), np.nan, dtype=float)
    
            # Create DataFrame for the current file
            df = pd.DataFrame(var_data)
            all_data.append(df)
        except Exception as e:
            print(f"Failed to process {path}: {e}")
    
    # Combine data from all files
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df.set_index('time', inplace=True)
        return combined_df
    else:
        return pd.DataFrame()

# %%
def process_line_p_data(start_year=1969, end_year=2020):
    path0 = '/gpfs/fs7/dfo/hpcmc/pfm/amh001/DATA/OUTPUTS_LINEP/CTD/'
    outpath = '/gpfs/fs7/dfo/hpcmc/pfm/amh001/DATA/OUTPUTS_LINEP/'
    figures_dir = 'Figures'
    
    # Create figures directory if it doesn't exist
    os.makedirs(figures_dir, exist_ok=True)
    
    def latlon_to_xyz(lat, lon):
        """Convert lat/lon (in radians) to 3D Cartesian coordinates."""
        R = 6371  # Earth radius in km
        x = R * np.cos(lat) * np.cos(lon)
        y = R * np.cos(lat) * np.sin(lon)
        z = R * np.sin(lat)
        return np.column_stack((x, y, z))
    
    print("Loading Line P station coordinates...")
    linep_df = pd.read_csv("/gpfs/fs7/dfo/hpcmc/pfm/amh001/DATA/SD-Ocean/observations_LineP/LineP.csv")
    linep_coords_rad = np.radians(linep_df[['latitude', 'longitude']].values)
    
    print("Building spatial index...")
    linep_xyz = latlon_to_xyz(linep_coords_rad[:, 0], linep_coords_rad[:, 1])
    tree = cKDTree(linep_xyz)
    
    station_names = linep_df['station_name'].values
    distance_thresholds = np.where(station_names == 'P26', 25.0, 10.0)
    
    for year in range(start_year, end_year + 1):
        print(f"\nProcessing year {year}...")
        nc_files = glob.glob(os.path.join(path0, f"*CastCTD_{year}*.nc"))
    
        if not nc_files:
            print(f"  No files found for year {year}")
            continue
    
        df_profiles = load_temperature_profiles(nc_files)
    
        if df_profiles.empty:
            print(f"  No valid profiles for year {year}")
            continue
    
        # Convert profile coordinates to xyz
        profile_coords_rad = np.radians(df_profiles[['latitude', 'longitude']].values)
        profile_xyz = latlon_to_xyz(profile_coords_rad[:, 0], profile_coords_rad[:, 1])
    
        # Query nearest stations
        distances, indices = tree.query(profile_xyz, k=1)
    
        df_profiles['closest_linep_station_name'] = station_names[indices]
        df_profiles['distance_to_closest_station_km'] = distances
    
        profile_thresholds = distance_thresholds[indices]
        kept_mask = distances < profile_thresholds
        kept_profiles = df_profiles[kept_mask]
        filtered_profiles = df_profiles[~kept_mask]
    
        # Save CSV
        if not kept_profiles.empty:
            output_filename = os.path.join(outpath, f'LineP_ctds_{year}.csv')
            kept_profiles.to_csv(output_filename, na_rep='NaN')
            
            station_counts = kept_profiles['closest_linep_station_name'].value_counts()
            print(f"  Saved {len(kept_profiles)} profiles for {year}:")
            for station, count in station_counts.items():
                threshold = 25 if station == 'P26' else 10
                print(f"    {station}: {count} profiles (within {threshold} km)")
        else:
            print(f"  No profiles within threshold distances for year {year}")
    
        # Generate map plot
        if not kept_profiles.empty or not filtered_profiles.empty:
            fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})
            ax.coastlines()
            ax.gridlines(draw_labels=False)
            
            ax.scatter(linep_df['longitude'], linep_df['latitude'],
                       color='blue', marker='x', s=50, linewidth=0.5, zorder=100,
                       label='Line P Stations')
            
            if not kept_profiles.empty:
                ax.scatter(kept_profiles['longitude'], kept_profiles['latitude'], 
                           facecolors='none', edgecolors='orange', s=20, alpha=0.7,
                           label=f'Kept Profiles ({len(kept_profiles)})')
            
            if not filtered_profiles.empty:
                ax.scatter(filtered_profiles['longitude'], filtered_profiles['latitude'],
                           facecolors='none', edgecolors='red', marker='^', s=20, alpha=0.7,
                           label=f'Eliminated Profiles ({len(filtered_profiles)})')
            
            ax.legend()
            ax.set_extent([-147, -123, 47, 52], crs=ccrs.PlateCarree())
            plot_filename = os.path.join(figures_dir, f'LineP_ctds_map_{year}.png')
            plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"  Saved plot to {plot_filename}")

# %%
process_line_p_data(start_year=1969, end_year=2020)

Loading Line P station coordinates...
Building spatial index...

Processing year 1969...
  Saved 10193 profiles for 1969:
    P26: 7002 profiles (within 25 km)
    P6: 442 profiles (within 10 km)
    P8: 411 profiles (within 10 km)
    P4: 364 profiles (within 10 km)
    P24: 307 profiles (within 10 km)
    P12: 298 profiles (within 10 km)
    P20: 297 profiles (within 10 km)
    P14: 256 profiles (within 10 km)
    P18: 253 profiles (within 10 km)
    P16: 200 profiles (within 10 km)
    P22: 189 profiles (within 10 km)
    P2: 117 profiles (within 10 km)
    P1: 57 profiles (within 10 km)
  Saved plot to Figures/LineP_ctds_map_1969.png

Processing year 1970...
  Saved 9065 profiles for 1970:
    P26: 6369 profiles (within 25 km)
    P6: 498 profiles (within 10 km)
    P8: 357 profiles (within 10 km)
    P4: 269 profiles (within 10 km)
    P12: 264 profiles (within 10 km)
    P18: 241 profiles (within 10 km)
    P20: 223 profiles (within 10 km)
    P24: 203 profiles (within 10 km)
   